In [ ]:
pip install torch_geometric

In [ ]:
pip install tensorboard

In [ ]:
import os
import imageio.v2 as imageio
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from skimage import io, segmentation
from torchvision import transforms
import torch.nn as nn
import numpy as np
import tifffile as tiff
from skimage.segmentation import slic, felzenszwalb, quickshift
from skimage.segmentation import mark_boundaries
from torchvision.transforms import functional as F
# import mahotas as mh
from skimage.measure import regionprops,label as label_image_regions

from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau


from tqdm import tqdm
from datetime import datetime
from timeit import default_timer as timer
import time

import networkx as nx
from torch_geometric.utils import grid, subgraph,to_networkx,select
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ConcatDataset, random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, PrecisionRecallDisplay
import seaborn as sns
import numpy as np
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from PIL import Image

In [ ]:
# from torch_geometric.data import Dataset, Data
# from torch_geometric.utils import grid, subgraph, select
# import os
# import imageio
# import torch
# import os.path as osp

# class PotsDamGraphDataset(Dataset):
#     def __init__(self, root_dir,out_dir, mode='train', transform=None):
#         self.root_dir = root_dir
#         self.mode = mode
#         self.transform = transform
#         self.images_folder = os.path.join(self.root_dir, f'images_{mode}')
#         self.meta_dir = os.path.join(self.root_dir, f'metadata_{mode}')
#         self.mask_dir = os.path.join(self.root_dir, f'masks_{mode}')

#         self.image_files = os.listdir(self.images_folder)
#         self.image_files.sort()

#         # Create directory for saving data based on mode
#         self.proc_dir = os.path.join(out_dir, f'processed_{mode}')
#         if not os.path.exists(self.proc_dir):
#             os.makedirs(self.proc_dir)

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_name = self.image_files[idx]
#         img_path = os.path.join(self.images_folder, img_name)
#         image = imageio.v2.imread(img_path)

#         # Extract index and segment index from the image name
#         sam_idx, _, _, segment_idx = img_name.split('_')
#         sam_idx = int(sam_idx)
#         segment_idx = int(segment_idx.split('.')[0])  # Remove '.tif' extension

#         # Load metadata
#         meta_filename = f"{sam_idx}_{self.mode}_metadata_{segment_idx}.txt"
#         meta_path = os.path.join(self.meta_dir, meta_filename)
#         with open(meta_path, 'r') as f:
#             metadata_lines = f.readlines()

#         # Extract label from the first line (index 1)
#         label = int(metadata_lines[0].split(':')[1])

#         # Extract centroid coordinates from the second line (index 1)
#         centroid_line = metadata_lines[1]
#         _, centroid_coords_str = centroid_line.split(': ')
#         centroid_coords_str = centroid_coords_str.strip()
#         centroid_coords_str = centroid_coords_str.replace('(', '').replace(')', '')
#         centroid_x_str, centroid_y_str = centroid_coords_str.split(', ')
#         centroid_coords = (float(centroid_x_str), float(centroid_y_str))
#         #centroid_coords = tuple(map(float, centroid_coords_str.split(',')))

#         # Load mask
#         mask_filename = f"{sam_idx}_{self.mode}_mask_{segment_idx}.tif"
#         mask_path = os.path.join(self.mask_dir, mask_filename)
#         mask = imageio.v2.imread(mask_path)
#         mask_tensor = torch.tensor(mask == True, dtype=torch.bool)

#         # Normalize image
#         normalized_image = image / 255.0
#         image_tensor = torch.tensor(normalized_image, dtype=torch.float)
#         centroid_tensor = torch.tensor(centroid_coords, dtype=torch.float)

#         # Compute edge indices and positions
#         height = image_tensor.shape[0]
#         width = image_tensor.shape[1]

#         image_grid = grid(height, width, dtype=torch.long)
#         edge_index, pos = image_grid

#         # Generate subgraphs
#         mask_idx = mask_tensor.flatten()
#         sub_graph_edge_index, _ = subgraph(mask_idx, edge_index, relabel_nodes=True)

#         # Get node features and positions corresponding to nodes in the subgraph
#         img = image_tensor.reshape(height * width, 4)
#         subgraph_img = select(img, mask_idx, dim=0)
#         subgraph_pos = select(pos, mask_idx, dim=0)

#         # Create Data object
#         data = Data(x=subgraph_img, edge_index=sub_graph_edge_index, y=label, pos=subgraph_pos)

#         # Save data to disk
#         data_path = osp.join(self.processed_dir, f'data_{idx}.pt')
#         torch.save(data, data_path)

#         return data


In [ ]:
# root_dir = '/kaggle/input/deepfeats-2/DeepFeats_Update2/' 
# out_dir= '/kaggle/working/'# Update this with the actual path to your data
# train_dataset = PotsDamGraphDataset(root_dir,out_dir, mode='train')


In [ ]:
# val_dataset = PotsDamGraphDataset(root_dir,out_dir, mode='val')

In [ ]:
# test_dataset = PotsDamGraphDataset(root_dir,out_dir, mode='test')

In [ ]:
class PotsDamGraphDataset(Dataset):
    def __init__(self, root_dir, mode='train', transform=None):
        self.root_dir = root_dir
        self.mode = mode
        self.transform = transform
        self.images_folder = os.path.join(self.root_dir, f'images_{mode}')
        self.meta_dir = os.path.join(self.root_dir, f'metadata_{mode}')
        self.mask_dir = os.path.join(self.root_dir, f'masks_{mode}')

        self.image_files = os.listdir(self.images_folder)
        self.image_files.sort()




    def __len__(self):
        return len(self.image_files)


    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_folder, img_name)
        image = imageio.imread(img_path)

        # Extract index and segment index from the image name
        sam_idx, _, _, segment_idx = img_name.split('_')
        sam_idx = int(sam_idx)
        segment_idx = int(segment_idx.split('.')[0])  # Remove '.tif' extension

        # Load metadata
        meta_filename = f"{sam_idx}_{self.mode}_metadata_{segment_idx}.txt"
        meta_path = os.path.join(self.meta_dir, meta_filename)
        with open(meta_path, 'r') as f:
            metadata_lines = f.readlines()

        # Extract label from the first line (index 1)
        label = int(metadata_lines[0].split(':')[1])
#         centroid = tuple(map(float, metadata[1].split(':')[1].split(',')))

        # Extract centroid coordinates from the second line (index 1)
        centroid_line = metadata_lines[1]

        # Split the line by colon and whitespace to isolate coordinate values
        _, centroid_coords_str = centroid_line.split(': ')
        centroid_coords_str = centroid_coords_str.strip()  # Remove leading/trailing whitespace

        # Remove parentheses from coordinate string and split by comma to separate x and y
        centroid_coords_str = centroid_coords_str.replace('(', '').replace(')', '')
        centroid_x_str, centroid_y_str = centroid_coords_str.split(', ')

        # Convert coordinate strings to floats and create a tuple
        centroid = (float(centroid_x_str), float(centroid_y_str))
        label_tensor = torch.tensor(label, dtype=torch.long)

        # Load mask
        mask_filename = f"{sam_idx}_{self.mode}_mask_{segment_idx}.tif"
        mask_path = os.path.join(self.mask_dir, mask_filename)
        mask = imageio.imread(mask_path)
        mask_tensor = torch.tensor(mask, dtype=torch.bool)

        # Normalize image
        normalized_image = image / 255.0
        image_tensor = torch.tensor(normalized_image, dtype=torch.float)
        centroid_tensor = torch.tensor(centroid, dtype=torch.float)


        sample = {'image': image_tensor, 'label': label_tensor,'mask': mask_tensor,'centroid':centroid_tensor}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
class RegionToGraph(nn.Module):
    def __init__(self):
        super(RegionToGraph, self).__init__()

    def forward(self,sample):
        image = sample['image']
        label = sample['label']
        mask = sample['mask']
        centroid = sample['centroid']
        #compute edge indices and positions
        height = image.shape[0]
        width = image.shape[1]

        image_grid = grid(height, width, dtype=torch.long)
        edge_index, pos = image_grid

        #generate subgraphs
        mask_idx=(mask == True).flatten()
        sub_graph_edge_index, _ = subgraph(mask_idx,edge_index,relabel_nodes=True)

        # Get node features and positions corresponding to nodes in the subgraph
        img=image.reshape(height*width,4)
        subgraph_img = select(img,mask_idx,dim=0)
        subgraph_pos = select(pos,mask_idx,dim=0)

        #compute pos relative to centroid
        #pos_rel = subgraph_pos-centroid

        # Concatenate 'subgraph_img' and 'pos_rel' along the feature dimension
        #node_feats = torch.cat([subgraph_img, pos_rel], dim=1)

        # Create Data object to represent the subgraph
        subgraph_data = Data(x=subgraph_img, edge_index=sub_graph_edge_index, y=label, pos=subgraph_pos)

        return subgraph_data


In [ ]:
# import os
# import torch

# class SaveGraphTransform(object):
#     def __init__(self, save_dir):
#         self.save_dir = save_dir

#         if not os.path.exists(self.save_dir):
#             os.makedirs(self.save_dir)

#     def __call__(self, subgraph_data):

#         # Define the file path to save the graph data
#         graph_path = os.path.join(self.save_dir, f'graph_{torch.randint(1000000, ())}.pt')

#         # Save the graph data
#         torch.save(subgraph_data, graph_path)

#         # Return the original sample
#         return sample


In [ ]:
root_dir = '/kaggle/input/deepfeats-2/DeepFeats_Update2/'

# Create the transform
transform = RegionToGraph()

mode='train'
train_dataset = PotsDamGraphDataset(root_dir, mode=mode,transform=transform)

In [ ]:
mode='test'
test_dataset = PotsDamGraphDataset(root_dir, mode=mode,transform=transform)

In [ ]:
mode='val'
val_dataset = PotsDamGraphDataset(root_dir, mode=mode,transform=transform)

In [ ]:
# Concatenate the train and validation datasets
combined_dataset = ConcatDataset([train_dataset, val_dataset])
# Load targets from the saved numpy file
targets = np.load('/kaggle/input/deepfeats-2/targets (2).npy')

# Define the sizes for the new splits
VAL_SIZE = 0.1
SEED = 42

# Perform the new split
train_indices, val_indices = train_test_split(
    range(len(combined_dataset)),
    test_size=VAL_SIZE,
    random_state=SEED,
    stratify=targets
)

# Generate subsets based on indices
new_train_dataset = Subset(combined_dataset, train_indices)
new_val_dataset = Subset(combined_dataset, val_indices)


In [ ]:
# Hyperparameters and device setup
LR = 0.001
epochs = 20
batch_size = 10
num_features = 4
num_classes = 7
out_channels = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
experiment = f'epochs_{epochs}_batch_size_{batch_size}_CNN_lr_{LR}'
log_dir = f'_{experiment}'
writer = SummaryWriter(comment=log_dir)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, pin_memory=True, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4, pin_memory=True, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, pin_memory=True, shuffle=False)

#or

train_loader = DataLoader(new_train_dataset, batch_size=batch_size,num_workers=4,pin_memory=True, shuffle=True,persistent_workers=True)
val_loader = DataLoader(new_val_dataset, batch_size=batch_size,num_workers=4,pin_memory=True, shuffle=False,persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,num_workers=4,pin_memory=True, shuffle=False,persistent_workers=True)

In [ ]:
# import torch
# import torch.nn.functional as F
# from torch.nn import Linear, BatchNorm1d
# from torch_geometric.nn import GraphConv, global_max_pool as gmp

# class GraphCon(torch.nn.Module):
#     def __init__(self, num_features, hidden_channels, num_classes):
#         super(GraphCon, self).__init__()
#         torch.manual_seed(12345)
        
#         self.conv1 = GraphConv(num_features, hidden_channels)
#         self.bn1 = BatchNorm1d(hidden_channels)
        
#         self.conv2 = GraphConv(hidden_channels, hidden_channels)
#         self.bn2 = BatchNorm1d(hidden_channels)
        
#         self.conv3 = GraphConv(hidden_channels, hidden_channels)
#         self.bn3 = BatchNorm1d(hidden_channels)
        
#         self.lin = Linear(hidden_channels, num_classes)

#     def forward(self, x, edge_index, batch):
#         x = self.conv1(x, edge_index)
#         x = self.bn1(x)
#         x = x.relu()
        
#         x = self.conv2(x, edge_index)
#         x = self.bn2(x)
#         x = x.relu()
        
#         x = self.conv3(x, edge_index)
#         x = self.bn3(x)

#         x = gmp(x, batch)  # Global max pooling

#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.lin(x)
        
#         return x




In [ ]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn import MessagePassing

class GCN(MessagePassing):
    def __init__(self, num_features, out_channels, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNConv(num_features, out_channels)
        self.gcn2 = GCNConv(out_channels, out_channels)
        self.gcn3 = GCNConv(out_channels, out_channels)
        
       
        # So the out_channels * 2 for the concatenated pooled features
        self.out = Linear(out_channels * 2, num_classes)

    def forward(self, x, edge_index, batch_index):
        x = self.gcn1(x, edge_index).relu()
        x = self.gcn2(x, edge_index).relu()
        x = self.gcn3(x, edge_index).relu()
        
        # Global Pooling (stack different aggregations)
        x = torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1)
        
        out = self.out(x)
        return x,out

In [ ]:
model = GCN(num_features=num_features, num_classes=num_classes, out_channels=out_channels).to(device)
print(model)
# model = model = GraphCon(num_features,hidden_channels=out_channels, num_classes=num_classes).to(device)
# print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = ReduceLROnPlateau(optimizer, mode='min')

In [ ]:


# # Wrap your main training loop in a function to ensure proper cleanup
# def train_model():
#     best_val_loss = float('inf')
#     patience = 5
#     no_improvement_counter = 0
#     all_true_labels = []
#     all_predicted_labels = []

#     try:
#         for epoch in range(epochs):
#             print(f'EPOCH {epoch + 1}:')
#             start_time = timer()
#             train_loss = 0.0
#             correct_predictions = 0
#             total_samples = 0

#             model.train()
#             for data in tqdm(train_loader):
#                 data = data.to(device)

#                 # Forward pass
#                 y_hat = model(data.x, data.edge_index, data.batch)

#                 # Compute the loss
#                 loss = criterion(y_hat, data.y)
#                 train_loss += loss.item()

#                 # Backward pass and optimization step
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()

#                 # Count correct predictions
#                 _, predicted = torch.max(y_hat, 1)
#                 correct_predictions += (predicted == data.y).sum().item()
#                 total_samples += data.y.size(0)

#             # Calculate accuracy and average loss for training
#             train_accuracy = correct_predictions / total_samples
#             train_average_loss = train_loss / total_samples

#             # Log training loss and accuracy to TensorBoard
#             writer.add_scalar('lr', optimizer.param_groups[0]['lr'], epoch + 1)
#             writer.add_scalar('loss/train', train_average_loss, epoch + 1)
#             writer.add_scalar('acc/train', train_accuracy, epoch + 1)

#             # Validation
#             model.eval()
#             valid_loss = 0.0
#             correct_predictions_val = 0
#             total_samples_val = 0

#             with torch.no_grad():
#                 for data_val in tqdm(val_loader):
#                     data_val = data_val.to(device)

#                     y_val_hat = model(data_val.x, data_val.edge_index, data_val.batch)

#                     val_loss = criterion(y_val_hat, data_val.y)
#                     valid_loss += val_loss.item()

#                     _, predicted_val = torch.max(y_val_hat, 1)
#                     correct_predictions_val += (predicted_val == data_val.y).sum().item()
#                     total_samples_val += data_val.y.size(0)

#                 scheduler.step(valid_loss)

#                 # Calculate accuracy and average loss for validation
#                 val_accuracy = correct_predictions_val / total_samples_val
#                 val_average_loss = valid_loss / total_samples_val

#                 # Log validation loss and accuracy to TensorBoard
#                 writer.add_scalar('loss/val', val_average_loss, epoch + 1)
#                 writer.add_scalar('acc/val', val_accuracy, epoch + 1)

#             # Reset lists to collect true labels and predictions for the test dataset
#             all_true_labels = []
#             all_predicted_labels = []

#             cum_test_loss = 0.0
#             correct_predictions_test = 0
#             total_samples_test = 0

#             with torch.no_grad():
#                 for data_test in tqdm(test_loader):
#                     data_test = data_test.to(device)

#                     y_test_hat = model(data_test.x, data_test.edge_index, data_test.batch)

#                     test_loss = criterion(y_test_hat, data_test.y)
#                     cum_test_loss += test_loss.item()

#                     _, predicted_test = torch.max(y_test_hat, 1)
#                     correct_predictions_test += (predicted_test == data_test.y).sum().item()
#                     total_samples_test += data_test.y.size(0)

#                     # Collect all true labels and predictions
#                     all_true_labels.extend(data_test.y.cpu().numpy())
#                     all_predicted_labels.extend(predicted_test.cpu().numpy())

#                 # Calculate accuracy and average loss for testing
#                 test_accuracy = correct_predictions_test / total_samples_test
#                 test_average_loss = cum_test_loss / total_samples_test

#                 # Log test loss and accuracy to TensorBoard
#                 writer.add_scalar('loss/test', test_average_loss, epoch + 1)
#                 writer.add_scalar('acc/test', test_accuracy, epoch + 1)

#             print(f'LOSS train {train_average_loss}, valid {val_average_loss}, test {test_average_loss}')
#             print(f'ACCURACY train {train_accuracy}, valid {val_accuracy}, test {test_accuracy}')

#             # Early stopping
#             if val_average_loss < best_val_loss:
#                 best_val_loss = val_average_loss
#                 no_improvement_counter = 0
#                 # Save the current best model
#                 best_model_path = f'./GAT_model_lr_{LR}.pth'
#                 torch.save({'epoch': epoch,
#                             'model_state': model.state_dict(),
#                             'optimizer_state_dict': optimizer.state_dict()}, best_model_path)
#             else:
#                 no_improvement_counter += 1

#             if patience is not None and no_improvement_counter >= patience:
#                 print(f"No improvement for {patience} epochs. Early stopping...")
#                 break

#             end_time = timer()
#             print(f"Epoch {epoch + 1} completed. Total epoch time: {end_time - start_time:.3f} seconds")

#     except Exception as e:
#         print(f"An error occurred: {e}")

#     finally:
#         # Generate classification report and confusion matrix
#         all_true_labels = np.array(all_true_labels)
#         all_predicted_labels = np.array(all_predicted_labels)
#         class_names = ['Imp_Surf', 'Bldgs', 'Low_Veg', 'Tree', 'Car', 'Clutter']

#         # Generate classification report
#         class_report = classification_report(all_true_labels, all_predicted_labels, target_names=class_names, zero_division=0)
#         print("Classification Report:\n", class_report)

#         # Generate confusion matrix
#         conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

#         # Plot confusion matrix
#         fig, ax = plt.subplots(figsize=(10, 8))
#         sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
#         ax.set_xlabel('Predicted Labels')
#         ax.set_ylabel('True Labels')
#         ax.set_title('Confusion Matrix')
#         plt.show()

#         # Optionally log classification report and confusion matrix to TensorBoard
#         writer.add_text('Classification Report', class_report)
#         fig_conf_matrix = plt.figure(figsize=(10, 8))
#         sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
#         plt.xlabel('Predicted Labels')
#         plt.ylabel('True Labels')
#         plt.title('Confusion Matrix')
#         writer.add_figure('Confusion Matrix', fig_conf_matrix)
#         plt.close(fig_conf_matrix)

# # Call the train_model function
# train_model()


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer
from tqdm import tqdm
import numpy as np

# Define the train_model function
def train_model():
    best_val_loss = float('inf')
    patience = 5
    no_improvement_counter = 0

    try:
        for epoch in range(epochs):
            print(f'EPOCH {epoch + 1}:')
            start_time = timer()
            train_loss = 0.0
            correct_predictions = 0
            total_samples = 0

            model.train()
            for data in tqdm(train_loader):
                data = data.to(device)

                # Forward pass
                y_hat = model(data.x, data.edge_index, data.batch)

                # Compute the loss
                loss = criterion(y_hat, data.y)
                train_loss += loss.item()

                # Backward pass and optimization step
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Count correct predictions
                _, predicted = torch.max(y_hat, 1)
                correct_predictions += (predicted == data.y).sum().item()
                total_samples += data.y.size(0)

            # Calculate accuracy and average loss for training
            train_accuracy = correct_predictions / total_samples
            train_average_loss = train_loss / total_samples

            # Log training loss and accuracy to TensorBoard
            writer.add_scalar('lr', optimizer.param_groups[0]['lr'], epoch + 1)
            writer.add_scalar('loss/train', train_average_loss, epoch + 1)
            writer.add_scalar('acc/train', train_accuracy, epoch + 1)

            # Validation
            model.eval()
            valid_loss = 0.0
            correct_predictions_val = 0
            total_samples_val = 0

            with torch.no_grad():
                for data_val in tqdm(val_loader):
                    data_val = data_val.to(device)

                    y_val_hat = model(data_val.x, data_val.edge_index, data_val.batch)

                    val_loss = criterion(y_val_hat, data_val.y)
                    valid_loss += val_loss.item()

                    _, predicted_val = torch.max(y_val_hat, 1)
                    correct_predictions_val += (predicted_val == data_val.y).sum().item()
                    total_samples_val += data_val.y.size(0)

                scheduler.step(valid_loss)

                # Calculate accuracy and average loss for validation
                val_accuracy = correct_predictions_val / total_samples_val
                val_average_loss = valid_loss / total_samples_val

                # Log validation loss and accuracy to TensorBoard
                writer.add_scalar('loss/val', val_average_loss, epoch + 1)
                writer.add_scalar('acc/val', val_accuracy, epoch + 1)

            print(f'LOSS train {train_average_loss}, valid {val_average_loss}')
            print(f'ACCURACY train {train_accuracy}, valid {val_accuracy}')

            # Early stopping
            if val_average_loss < best_val_loss:
                best_val_loss = val_average_loss
                no_improvement_counter = 0
                # Save the current best model
                best_model_path = f'./GAT_model_lr_{LR}.pth'
                torch.save({'epoch': epoch,
                            'model_state': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict()}, best_model_path)
            else:
                no_improvement_counter += 1

            if patience is not None and no_improvement_counter >= patience:
                print(f"No improvement for {patience} epochs. Early stopping...")
                break

            end_time = timer()
            print(f"Epoch {epoch + 1} completed. Total epoch time: {end_time - start_time:.3f} seconds")

    except Exception as e:
        print(f"An error occurred: {e}")

# Define the test_model function
def test_model():
    # Load the best saved model
    checkpoint = torch.load(f'./GAT_model_lr_{LR}.pth')
    model.load_state_dict(checkpoint['model_state'])
    model.eval()

    all_true_labels = []
    all_predicted_labels = []
    cum_test_loss = 0.0
    correct_predictions_test = 0
    total_samples_test = 0

    with torch.no_grad():
        for data_test in tqdm(test_loader):
            data_test = data_test.to(device)

            y_test_hat = model(data_test.x, data_test.edge_index, data_test.batch)

            test_loss = criterion(y_test_hat, data_test.y)
            cum_test_loss += test_loss.item()

            _, predicted_test = torch.max(y_test_hat, 1)
            correct_predictions_test += (predicted_test == data_test.y).sum().item()
            total_samples_test += data_test.y.size(0)

            # Collect all true labels and predictions
            all_true_labels.extend(data_test.y.cpu().numpy())
            all_predicted_labels.extend(predicted_test.cpu().numpy())

        # Calculate accuracy and average loss for testing
        test_accuracy = correct_predictions_test / total_samples_test
        test_average_loss = cum_test_loss / total_samples_test

        # Log test loss and accuracy to TensorBoard
        writer.add_scalar('loss/test', test_average_loss, epoch + 1)
        writer.add_scalar('acc/test', test_accuracy, epoch + 1)

    print(f'LOSS test {test_average_loss}')
    print(f'ACCURACY test {test_accuracy}')

    # Generate classification report and confusion matrix
    all_true_labels = np.array(all_true_labels)
    all_predicted_labels = np.array(all_predicted_labels)
    class_names = ['Imp_Surf', 'Bldgs', 'Low_Veg', 'Tree', 'Car', 'Clutter']

    # Generate classification report
    class_report = classification_report(all_true_labels, all_predicted_labels, target_names=class_names, zero_division=0)
    print("Classification Report:\n", class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

    # Plot confusion matrix
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')
    ax.set_title('Confusion Matrix')
    plt.show()

    # Optionally log classification report and confusion matrix to TensorBoard
    writer.add_text('Classification Report', class_report)
    fig_conf_matrix = plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    writer.add_figure('Confusion Matrix', fig_conf_matrix)
    plt.close(fig_conf_matrix)



In [ ]:

# Call the train_model function
train_model()



In [ ]:
# Call the test_model function
test_model()

In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm

def infer_model():
    all_true_labels = []
    all_predicted_labels = []
    cum_test_loss = 0.0
    correct_predictions_test = 0
    total_samples_test = 0

    # Load the best model
    best_model_path = f'./GAT_model_lr_{LR}.pth'
    checkpoint = torch.load(best_model_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state'])
    model.to(device)
    model.eval()

    with torch.no_grad():
        for data_test in tqdm(test_loader):
            data_test = data_test.to(device)

            y_test_hat = model(data_test.x, data_test.edge_index, data_test.batch)

            test_loss = criterion(y_test_hat, data_test.y)
            cum_test_loss += test_loss.item()

            _, predicted_test = torch.max(y_test_hat, 1)
            correct_predictions_test += (predicted_test == data_test.y).sum().item()
            total_samples_test += data_test.y.size(0)

            # Collect all true labels and predictions
            all_true_labels.extend(data_test.y.cpu().numpy())
            all_predicted_labels.extend(predicted_test.cpu().numpy())

    # Calculate accuracy and average loss for testing
    test_accuracy = correct_predictions_test / total_samples_test
    test_average_loss = cum_test_loss / total_samples_test

    # Log test loss and accuracy to TensorBoard
    writer.add_scalar('loss/test', test_average_loss, 0)
    writer.add_scalar('acc/test', test_accuracy, 0)

    print(f'LOSS test {test_average_loss}')
    print(f'ACCURACY test {test_accuracy}')

    # Generate classification report and confusion matrix
    all_true_labels = np.array(all_true_labels)
    all_predicted_labels = np.array(all_predicted_labels)
    class_names = ['Imp_Surf', 'Bldgs', 'Low_Veg', 'Tree', 'Car', 'Clutter']

    # Generate classification report
    class_report = classification_report(all_true_labels, all_predicted_labels, target_names=class_names, zero_division=0)
    print("Classification Report:\n", class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

    # Plot confusion matrix
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')
    ax.set_title('Confusion Matrix')
    plt.show()

    # Optionally log classification report and confusion matrix to TensorBoard
    writer.add_text('Classification Report', class_report)
    fig_conf_matrix = plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    writer.add_figure('Confusion Matrix', fig_conf_matrix)
    plt.close(fig_conf_matrix)

# Call the infer_model function
infer_model()
